# Helper Functions for Showing how to Detect ARCH Effects

- (i) lag_data(.): generates y and x consistent with a specific AR(lag) structure

$$
AND
$$
- (ii) ljung_box(.) also known as PortmanteauTest.

In [1]:
#necessary packages
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy import stats

**(i): Generate lagged data from existing data**

In [1]:
def lag_data(data, lags):
    nobs = data.shape[0]
    
    y_lag = data[lags : nobs]
    x_lag = np.matrix(np.zeros( (nobs-lags) * lags).
                      reshape( (nobs-lags), lags) )
    
    for i in range(1, lags+1):
        x_lag[:,i-1] = data[ (lags-i) : (nobs-i) ].\
            reshape(nobs-lags, 1)
        
    return x_lag, y_lag

**(ii): Portmanteau Test (i.e. Ljung-Box test) of AR(p) residulas to search for ARCH(m) effects**

$$ H_0: \rho_1 = \rho_2 = ... = \rho_m = 0$$
$$ H_1 : \rho_l \neq 0\ \textrm{for some}\ l \in \{1,...,m\}$$
$$ Q(m) = T(T+2)\sum_{l=1}^{m} \frac{\hat{\rho}_l^2}{T-l} \overset{a}{\sim} \chi^2(m)$$
where $\rho_l$ is the l-th auto-correlation in the time series of squared residuals.


In [ ]:
def Portmanteau_Test(data, p, m): 

    # Run OLS regression to obtain residuals of AR(p) model
    #
    Y = lag_data(data, p)[1]
    X = sm.add_constant(lag_data(data, p)[0])
    reg = sm.OLS(Y, X).fit()
    resid = reg.resid

    T = len(resid)

    # Calculate auto-correlations in the squared residuals up to lag m
    #
    eps2_t = resid**2
    # Iterate across all lags l up to m and calculate test-statistics Q(m)
    #
    total = 0

    for l in range(1, m+1):
        rho2 = pd.Series(eps2_t).autocorr(l)**2
        total = total + rho2 / (T - l)

    Q = T * (T+2) * total

    pval = stats.chi2.pdf(Q, m)

    return Q, pval